# BERT Transfer Learning for Sentiment Analysis
**Objective:** Provide a framework to perform transfer learning using the pre-trained distilBERT model, 
allowing options for fine-tuning the distilBERT model or simply use its outputs as features. In this example, 
we use a dataset of Yelp reviews and build a sentiment classifier to identify whether a 
review is 1 or 5 stars.

In [3]:
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
import torch
import numpy as np
import json
from tqdm import tqdm
device = 'cpu'
device = 'gpu'

# Data pre-processing
**Summary:** parse Yelp reviews for the review text and number of stars associated with that review.
Only parse reviews with low or high stars, and ensure that we have an equal number of low and high star reviews. Low star reviews have 1 or 2 stars, and high star reviews have 5 stars.<br /> 
- *lowstar_review_limit*: once we parse this number of low star reviews have been parsed. Break from processing. Typically there are more high star than low star reviews so the total number of reviews read in will be twice this number. <br /> 
- *review_limit*: once a total of this number of reviews have been parsed, stop reading in more.<br /> 
- *sample_per_cat*: sample this many low star and high star reviews respectively from what is parsed.<br /> 
- *max_num_words*: only parse reviews with number of words less than this length <br /> 
[Download the data](https://www.yelp.com/dataset/documentation/main) <br /> 

In [4]:
#read-in data
lowstar_review_limit = 1024
review_limit = np.inf
sample_per_cat = 1024
max_num_words = 50


path = "/home/pd/datasets/yelp_reviews/yelp_reviews.json"
review_fields_wanted = ['text','lowstar']
rev = pd.DataFrame(columns=review_fields_wanted)
with open(path,encoding='utf-8') as d:
    counter = 0
    lowstar_counter = 0
    for line in d:
        L = json.loads(line)
        lowstar = L['stars'] == 1 or (L['stars'] == 2)
        fivestar = L['stars'] == 5
        not1or5 = not(lowstar or fivestar)
        if len(L['text'].split()) > max_num_words or not1or5:
            continue
        if lowstar:
            lowstar_counter += 1
            L['lowstar'] = 1
        else:
            L['lowstar'] = 0
        less_fields = {key: L[key] for key in review_fields_wanted }
        rev.loc[counter] = less_fields
        counter += 1
        if counter == review_limit or lowstar_counter == lowstar_review_limit:
            break

            
rev = rev.rename(columns = {'text':'_text','lowstar':'_lowstar'})


rev = rev.groupby('_lowstar').apply(lambda x: x.sample(sample_per_cat)).reset_index(drop=True)
rev['TARGETS'] = rev['_lowstar']
print(f'Number of 1 star reviews:{rev._lowstar[rev._lowstar == 1].count()}')
print(f'Number of 5 star reviews:{rev._lowstar[rev._lowstar == 0].count()}')
print(rev._text[rev._lowstar == 1].sample(5))
print(rev._text[rev._lowstar == 0].sample(5))


Number of 1 star reviews:1024
Number of 5 star reviews:1024
1142    Def,, not a place I\nGot\nFor\nItalian\nFood e...
1715    Bad service, always out of things and rude whe...
1070    I order the bone in ribeye fit market value. T...
1284    I wish this place was more awesome. I love han...
1397    I came here the other day and the person that ...
Name: _text, dtype: object
774    Best place late night.. authentic simple food....
595    Finding out that your home will need new drain...
915    Great food. Great atmosphere. The decor is rea...
958    I visited the new Paoli location. BIG selectio...
188    This place is fantastic! The ambiance is like ...
Name: _text, dtype: object


In [5]:
#option to save the parsed reviews as a csv
rev.to_csv('/home/pd/datasets/yelp_reviews/yelp_review_2048.csv')

# Pytorch dataset definition

In [29]:
#def dataset
class DFToTokenized(Dataset):
    def __init__(self,df,tokenizer,max_len):
        self.len = len(df)
        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self,index):

        review = ' '.join(self.data['_text'][index].split())
        inp = self.tokenizer.encode_plus(
            review,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        tokens = inp['input_ids']
        mask = inp['attention_mask']

        return {
            'ids': torch.tensor(tokens, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.TARGETS[index], dtype=torch.uint8)
        } 

    def __len__(self):
        return self.len
        



# Training params and dataloader
- *MAX_LEN*: only allow reviews with token length less than this to be used. The
token length is dictated by tokenization using the distilBERT vocabulary. Padding 
will then be performed to *max_len*.
- *AUTO_SCALE_GRAD*: if true, this will automatically scale the precision of floats
involved in gradient calculation to optimize performance.

In [42]:
#init train/test params
MAX_LEN = 64
TRAIN_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 1e-05
AUTO_SCALE_GRAD = False
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

train_frac = 0.8
train_dataset=rev.sample(frac=train_frac,random_state=200)
test_dataset=rev.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

training_set = DFToTokenized(train_dataset, tokenizer, MAX_LEN)
testing_set = DFToTokenized(test_dataset, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

single_params = {'batch_size': 1,
                'shuffle': True,
                'num_workers': 0
}

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

single_loader = DataLoader(training_set,**single_params)

# Model definition
distilBERT encoder's CLS token's hidden state is fed forward to eventually 
classify into positive or negative sentiment.

In [31]:
class DBertMultiCat(torch.nn.Module):
    def __init__(self):
        super(DBertMultiCat, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [32]:
#INIT model, loss, optimizer
model = DBertMultiCat()
for p in model.l1.parameters():
    p.requires_grad = False
model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
params_with_grad = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(params =  params_with_grad, lr=LEARNING_RATE)
if AUTO_SCALE_GRAD:
    scaler = torch.cuda.amp.GradScaler()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Training loop

In [33]:
#train loop def
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0),total=len(training_loader),
        position=0, leave=True):

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.uint8)

        if AUTO_SCALE_GRAD:
            with torch.cpu.amp.autocast():
                outputs = model(ids, mask)
                loss = loss_function(outputs, targets)
        else:
            outputs = model(ids, mask)
            loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")


        optimizer.zero_grad(set_to_none=True)
        if(AUTO_SCALE_GRAD):
            scaler.scale(loss).backward()
            # # When using GPU
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [35]:
#Run the training
for epoch in range(30):
    train(epoch)

  4%|▍         | 1/26 [00:02<00:55,  2.21s/it]

Training Loss per 5000 steps: 0.687899649143219
Training Accuracy per 5000 steps: 62.5


100%|██████████| 26/26 [01:02<00:00,  2.40s/it]


The Total Accuracy for Epoch 0: 52.747252747252745
Training Loss Epoch: 0.6927502017754775
Training Accuracy Epoch: 52.747252747252745


  4%|▍         | 1/26 [00:02<01:06,  2.67s/it]

...

The Total Accuracy for Epoch 28: 62.27106227106227
Training Loss Epoch: 0.6637566593977121
Training Accuracy Epoch: 62.27106227106227


  4%|▍         | 1/26 [00:23<09:54, 23.76s/it]

Training Loss per 5000 steps: 0.7042104601860046
Training Accuracy per 5000 steps: 50.0


100%|██████████| 26/26 [07:04<00:00, 16.34s/it]

The Total Accuracy for Epoch 29: 59.70695970695971
Training Loss Epoch: 0.666911624945127
Training Accuracy Epoch: 59.70695970695971


# Validation loop definition

In [51]:
#validation def
VALID_BATCH_SIZE = 1 #set to 1 for printing of individual wrong predictions

#detokenize def
def DBDetokenize(a):
    a_orig = [tokenizer.decode(x) for x in a['ids'].squeeze().tolist() if x != 0]
    a_orig = ([x.replace(' ' , '') for x in a_orig])
    return " ".join(a_orig)

def valid(model, testing_loader):
    tr_loss = 0 #added
    nb_tr_steps = 0 #added
    nb_tr_examples = 0 #added
    max_wrong_outputs = 10
    wrong_outputs = 0
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask)#.squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            #print individual wrong responses to file
            if VALID_BATCH_SIZE == 1 and wrong_outputs < max_wrong_outputs: 
                wrong_outputs += 1
                path = '/home/pd/summaries/yelp_summary_13Mar23.txt'
                with open(path,'a') as f:
                    f.write(DBDetokenize(data))
                    f.write(f'Should be: {1 if targets.item() else 5}')
                    f.write('\n')
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

In [52]:
#validation run
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

/home/pd/NNBasics/venv_NNBasics/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Validation Loss per 100 steps: 0.4546707570552826
Validation Accuracy per 100 steps: 100.0
Validation Loss Epoch: 0.6451624179395233
Validation Accuracy Epoch: 64.8780487804878
Accuracy on test data = 64.88%


In [37]:
output_model_file = '/home/pd/models/yelp_sentiment.bin'
output_vocab_file = '/home/pd/models/yelp_sentiment_vocab.bin'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

('/home/pd/models/yelp_sentiment_vocab.bin',)